In [161]:
import requests
import re
import time
import pandas as pd

from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression

In [172]:
url_dict = {
    'https://www.azlyrics.com':('https://www.azlyrics.com/d/dylan.html','listalbum-item','ringtone'),
    'https://www.lyrics.com':('https://www.lyrics.com/artist/Bob-Dylan/','tal qx','lyric-body')
}

In [168]:
class BobDylan:
    def __init__(self):
        self.url_1 = 'https://www.azlyrics.com/d/dylan.html'
        self.cls_1 = 'listalbum-item'
        self.subcls_1 = 'ringtone'
        self.url_2 = 'https://www.lyrics.com/artist/Bob-Dylan/'
        self.cls_2 = 'tal qx'
        self.subcls_2 = 'lyric-body'
        self.headers = {"User-Agent": 
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36"}


    def get_song_links(self):
        response = requests.get(self.url_2,headers=self.headers).text
        time.sleep(10)
        soup = BeautifulSoup(response)
        links = []
        soup_links = soup.find_all(class_ = self.cls_2)
        for link in soup_links:
            if link.a:
                ln = link.a['href']
                links.append(f'https://www.lyrics.com{ln}')
        return links
    
    def get_lyrics(self,link):       
        response = requests.get(link,headers=self.headers)
        time.sleep(10) 
        if response.status_code == 404: print('blocked')
        soup = BeautifulSoup(response.text)
        lyrics = soup.find(class_ = self.subcls_2) #
        return lyrics.text

    def get_song_links_alyrics(self):
        response = requests.get(self.url_1,headers=self.headers)
        time.sleep(10)
        soup = BeautifulSoup(response)
        links = []
        soup_links = soup.find_all(class_ = self.cls_1)
        for link in soup_links:
            if link.a:
                ln = link.a['href']
                links.append(f'https://www.azlyrics.com{ln}')
        return links

    def get_lyrics_alyrics(self,link):       
        response = requests.get(link,headers=self.headers)
        time.sleep(10) 
        if response.status_code == 404: print('blocked')
        soup = BeautifulSoup(response.text)
        lyrics = soup.find(class_ = self.subcls_2) #
        return lyrics.text


In [163]:
Bobdylan = BobDylan()

In [169]:
Bobdylan_links = Bobdylan.get_song_links()

In [165]:
all_lyrics = []
for link in Bobdylan_links[0:2]:
    lyrics = Bobdylan.get_lyrics(link)
    with open(f'./BobDylan_lyrics.txt','a') as my_file:
        my_file.write(lyrics)    
    all_lyrics.extend(lyrics.replace('\r','').split('\n'))

In [166]:
all_lyrics.__sizeof__()

776

In [167]:
vectorizer = CountVectorizer(lowercase=True, token_pattern='[A-Za-z]+', ngram_range=(1,1))
X_cv = vectorizer.fit_transform(all_lyrics)
df_bow_sklearn = pd.DataFrame(X_cv.toarray(), columns=vectorizer.get_feature_names_out())
df_bow_sklearn

,a,about,again,ain,all,alone,always,an,and,another,...,wilderness,will,with,without,word,world,wrists,ya,you,your
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
88,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
90,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
